# Tp LoRa

L'objectif de ce TP est d'apprendre à utiliser la technologie LoRa avec la carte LoPy4 de Pycom. Nous allons configurer et tester deux scénarios :

Envoi de données vers The Things Network (TTN).
Communication directe entre deux cartes LoPy4.

Matériel requis :
- 2 cartes LoPy4 de Pycom
- 2 antennes LoRa compatibles
- 2 batteries ou câbles USB pour l'alimentation
- 1 ordinateur avec Visual Studio Code et l'extension PyMakr installée


## envoi des données vers Tne Things Network

### Configuration de la passerelle

(partie a réaliser uniquement si la passerelle n'est pas deja configurée)

Pour configurer la passerelle, il faut etre sur le même réseau et se rendre a l'adresse 147.127.245.250
- Username : admin
- Password : centrale2022

<img src="../img/gw_connexion.png" alt="Drawing" style="width: 600px;"/>

Configurer les paramètres de réseau en mode passerelle 
- LoraWan -> Network Settings
    - mode : packet Forwarder
    - Channel Plan : EU868

<img src="../img/gw_paramNet.png" alt="Drawing" style="width: 600px;"/>




### Connexion avec The Things Network

Connectez votre carte LoPy4 à votre ordinateur via USB.

Accedez au site https://eu1.cloud.thethings.network/console/
- id :  duiot
- mdp : duiot2020

Pour vérifier que la passerelle es bien configurée cliquez sur gateway, ici, on peut voir si la passerelle est connectée ou non.

<img src="../img/ttn_gw.png" alt="Drawing" style="width: 600px;"/>

Occupons nous maintenant de créer notre application
- cliquez sur applications

<img src="../img/ttn_cnx.png" alt="Drawing" style="width: 600px;"/>

Vous avez maintenant la liste des applications déjà crées, créez votre propre application
- cliquez sur "create application"

<img src="../img/ttn_app.png" alt="Drawing" style="width: 600px;"/>

Vous allez donc renseigner toutes les informations de votre application 
- remplir les infos de l'application

<img src="../img/ttn_createApp.png" alt="Drawing" style="width: 600px;"/>

Une fois dans l'application, vous devez enregistrer les appareils liés à l'application
- cliquez sur "Register end device"

<img src="../img/ttn_registerDevice.png" alt="Drawing" style="width: 600px;"/>

Vous devez ensuite rentrer les informations de votre appareil pour que celui-ci soit reconnu par l'application
- cliquez sur "Enter end device specifics manually"
- renseignez les informations de votre carte 
    - Europe 863-870 MHz (SF9 for RX2 - recommended)
    - LoRaWAN Specification 1.0.0
    - TS001 Technical Specification 1.0.0
    - AppEUI, DevEUI, ... voir le tableur sur moodle 

<img src="../img/ttn_appCree.png" alt="Drawing" style="width: 600px;"/>

Vous pouvez maintenant avoir accès au panel de votre application, ici, vous pouvez voir toutes les communications et informations de votre application.
Sur le volet de gauche vous pouvez aussi avoir accès a différents onglets qui détaillent ces informations, vous avez aussi la possibilité de générer une clé API pour récupérer les données recues par TTN
<img src="../img/ttn_appPanel.png" alt="Drawing" style="width: 600px;"/>



### code du LoPy4

Sur Vs Code, l'extension pymakr doit etre installée.

Le code suivant permet de connaitre le statut de la connexion avec the things network
- ouvrir le dosier /home/local/Documents/StageIoT/pymakr/project dans vs code
- le code suivant doit etre dans main.py
    - penser à changer app_eui, app_key et dev_eui par les valeurs rentrées dans The Things Network

In [ ]:
from network import LoRa
import socket
import time
import ubinascii
import pycom

pycom.heartbeat(False)

# Initialise LoRa in LORAWAN mode.
# Please pick the region that matches where you are using the device:
# Asia = LoRa.AS923
# Australia = LoRa.AU915
# Europe = LoRa.EU868
# United States = LoRa.US915
lora = LoRa(mode=LoRa.LORAWAN, region=LoRa.EU868)

# create an OTAA authentication parameters, change them to the provided credentials
app_eui = ubinascii.unhexlify('70B3D57ED0038811')
app_key = ubinascii.unhexlify('583FE2F370E3F43BCFE06291DCD155A1')
#uncomment to use LoRaWAN application provided dev_eui
dev_eui = ubinascii.unhexlify('70b3d5499e370b3d')

# join a network using OTAA (Over the Air Activation)
#uncomment below to use LoRaWAN application provided dev_eui
#lora.join(activation=LoRa.OTAA, auth=(app_eui, app_key), timeout=0)
lora.join(activation=LoRa.OTAA, auth=(dev_eui, app_eui, app_key), timeout=0)

# wait until the module has joined the network
while not lora.has_joined():
    pycom.rgbled(0x101000)
    time.sleep(0.3)
    pycom.rgbled(0x000000)
    time.sleep(0.3)
    

print('Joined')
pycom.rgbled(0x001000)
# create a LoRa socket
s = socket.socket(socket.AF_LORA, socket.SOCK_RAW)

# set the LoRaWAN data rate
s.setsockopt(socket.SOL_LORA, socket.SO_DR, 5)

# make the socket blocking
# (waits for the data to be sent and for the 2 receive windows to expire)


# send some data
while 1 :
    s.setblocking(True)
    
    s.send(bytes("1","utf-8"))

    # make the socket non-blocking
    # (because if there's no data received it will block forever...)
    s.setblocking(False)

    # get any data received (if any...)
    data = s.recv(64)
    if data :
        print(data)
    time.sleep(1)
    

### Téléversement du Code sur la Carte

#### Méthode 1 :

- Aller dans l'extension PyMakr.
- Cliquer sur ADD DEVICES.
- Choisir ttyACM0 (le port de la carte).

<img src="../img/code_connectPyCom.png" alt="Drawing" style="width: 300px;"/>

- Cliquer sur la petite icône d'éclair en passant la souris sur le device ajouté pour connecter la carte.

<img src="../img/code_connectDevice.png" alt="Drawing" style="width: 300px;"/>

- Envoyer les fichiers sur la carte en cliquant sur Sync project to device (si une erreur apparaît en bas à droite, passez à la méthode 2).

<img src="../img/code_sync.png" alt="Drawing" style="width: 300px;"/>

- Vous pouvez aussi ouvrir un terminal pour voir ce qui se passe sur la carte (optionnel).

#### Méthode 2 :

- Aller dans l'extension PyMakr.
- Cliquer sur ADD DEVICES.
- Choisir ttyACM0 (le port de la carte).

<img src="../img/code_connectPyCom.png" alt="Drawing" style="width: 300px;"/>

- Cliquer sur la petite icône d'éclair en passant la souris sur le device ajouté pour connecter la carte.

<img src="../img/code_connectDevice.png" alt="Drawing" style="width: 300px;"/>

- Vous pouvez ouvrir un terminal pour voir ce qui se passe sur la carte (optionnel).
- Retourner dans l'onglet fichiers.
- Faire un clic droit sur main.py.

<img src="../img/code_onDevice.png" alt="Drawing" style="width: 300px;"/>

- Aller sur PyMakr et sélectionner Run file on device.

#### Indications LED:

- Si la LED clignote en bleu, essayez de cliquer sur le bouton de la carte et recommencez le téléversement.
- Si la LED clignote en orange, le code est bien téléversé mais il y a un problème dans votre configuration LoRa.
- Si la LED est verte, tout fonctionne.

##